In [1]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence

import urllib.request, json
import numpy as np

2024-08-01 11:54:01.632575: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 11:54:01.667599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 11:54:01.667624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 11:54:01.668675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 11:54:01.675279: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 11:54:01.675761: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
res = urllib.request.urlopen("http://127.0.0.1:8000/api/vagas?treinamento=true")
data = json.loads(res.read())
train_vagas_data = data['train']
test_vagas_data = data['test']

train_X = []
y_train = []

test_X = []
y_test = []

for i in train_vagas_data:
    train_X.append(i['txtVaga'].replace("\n", " ").lower())
    y_train.append(int(i['isApplied']))
    
    
for i in test_vagas_data:
    test_X.append(i['txtVaga'].replace("\n", " ").lower())
    y_test.append(int(i['isApplied']))

char_set = ['<pad>'] + sorted(list(set(''.join(train_X + test_X)))) + ['<_unknow']
idx2char = {idx: char for idx, char in enumerate(char_set)}
char2idx = {char: idx for idx, char in enumerate(char_set)}

json_str = json.dumps(char2idx, ensure_ascii=False)
json_str = json.loads(json_str)
with open('char2idx.json', 'w', encoding='utf-8') as json_file:
    json.dump(json_str, json_file, ensure_ascii=False)
    
    
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = list(map(lambda sentence: [char2idx.get(char) for char in sentence], train_X))
X_test = list(map(lambda sentence: [char2idx.get(char) for char in sentence], test_X))


In [3]:
max_review_length = 200
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [4]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(len(char2idx), embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=32)

2024-08-01 11:54:05.603650: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/2
2/2 [==============================] - 2s 458ms/step - loss: 0.6934 - accuracy: 0.4412 - val_loss: 0.6910 - val_accuracy: 0.5000
Epoch 2/2
2/2 [==============================] - 0s 84ms/step - loss: 0.6920 - accuracy: 0.5294 - val_loss: 0.6906 - val_accuracy: 0.7500


In [8]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

[0.6906152963638306, 0.75]
Accuracy: 75.00%


In [6]:
pred = model.predict(X_test)
print(pred)

1/1 [==============================] - 0s 285ms/step
[[0.50009435]
 [0.5004008 ]
 [0.5056968 ]
 [0.505695  ]
 [0.5025173 ]
 [0.5025187 ]
 [0.49830014]
 [0.49834016]]


In [7]:
model.save("model.keras")